# Подготовка итоговой модели и данных для предсказания ожидаемой продолжительности жизни в странах

На основе проведенных исследований и созданных признаков подготовим итоговую модель для предсказания ожидаемой продолжительности жизни в отдельных странах после 2020 г.\
Будем делать предсказание за 2021 - 2023 гг.

Возьмем за основу данные за 2000-2020 гг. и добавим признаки, которые хорошо себя показали.

Сформируем таблицы аналогично, как для итоговой модели в файле с предсказанием [../05_prediction/01_main.ipynb](../05_prediction/01_main.ipynb) .\
И сохраним модель, которая показала наилучший результат, для дальнейшего использования.

In [5]:
import sys
# Добавим папку с корнем проекта в список системных директорий, чтобы Python видел путь к папке utils
sys.path.append('..')

In [6]:
from IPython.display import Markdown
from typing import Tuple
import time
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import  Lasso
from sklearn.ensemble import (
    AdaBoostRegressor,
)
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import pickle
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

from utils.constants import (
    F, 
    RANDOM_STATE,
    TEST_YEARS_COUNT,
)
from utils.helpers import get_exec_time
from utils.display_content import (
    display_fields_correlation,
)
from utils.prepare_data import (
    get_location_codes,
    get_location_time_series,
    get_location_by_code,
    get_train_test_data,
    get_data_with_smoothing_target_feature,
    get_integration_order,
    get_predictors,
    get_train_test_split,
    get_location_data,
    get_formatted_time_series,
    move_column_to_end_table,
)

%matplotlib inline

from warnings import simplefilter
simplefilter('ignore')

In [7]:
# Загрузим таблицу
data = pd.read_csv(
    '../data/cumulative_life_expectancy_prepared.csv'
)

data.head()

,ParentLocationCode,ParentLocation,SpatialDimValueCode,Location,Period,AdultMortality,Homicides,MaternalMortality,AdultNcdMortality,AdultNcdMortality117,...,Sanitation,DrinkingWater,HealthCareCosts,HealthCareCostsGdp,HealthCareCostsPerCapita,GdpPerCapita,Population,Schooling,ImmunizationMean,LifeExpectancy
0,AFR,Africa,AGO,Angola,2000,34.56,0.01331,0.859921,30.5,0.028126,...,27.56,41.14,2.74,1.11,7.66,563.733796,16194869.0,5.027188,28.000000,49.37
1,AFR,Africa,AGO,Angola,2001,33.86,0.01310,0.799641,29.7,0.027503,...,28.99,42.25,5.97,2.05,13.35,533.586202,16747208.0,5.092250,43.333333,50.06
2,AFR,Africa,AGO,Angola,2002,32.51,0.01288,0.758272,29.4,0.027049,...,30.42,43.38,4.19,1.31,11.54,882.147847,17327699.0,5.157312,42.666667,51.06
3,AFR,Africa,AGO,Angola,2003,32.16,0.01265,0.676496,29.3,0.026962,...,31.86,44.36,4.26,1.47,14.55,992.698979,17943712.0,5.222375,37.000000,51.74
4,AFR,Africa,AGO,Angola,2004,32.24,0.01229,0.594192,29.0,0.026300,...,33.29,45.35,5.66,1.69,21.36,1266.210864,18600423.0,5.287437,36.000000,52.36


In [8]:
# Загружаем таблицу с данными о странах и регионах
regions_and_locations_data = pd.read_csv(
    '../data/regions_and_locations.csv'
)

regions_and_locations_data.head()

,ParentLocationCode,ParentLocation,SpatialDimValueCode,Location
0,AFR,Africa,AGO,Angola
1,AFR,Africa,BDI,Burundi
2,AFR,Africa,BEN,Benin
3,AFR,Africa,BFA,Burkina Faso
4,AFR,Africa,BWA,Botswana


---

## Создание таблицы с данными стран для прогноза

Далее составим таблицу с данными стран за годы, для которых будем делать предсказание.\
Эта таблица будет нужна для создания интерфейса, с помощью которого можно будет сделать предсказание для отдельной страны и посмотреть на результат.\
Предикторы заполним сглаженными значениями на основе предыдущих лет.\
Исключение составит константа NegativeCoef, которая одинакова для всех лет.

In [187]:
# Определим список лет для предсказания
start_prediction_year = data[F.Period.value].max() + 1
prediction_years = list(range(start_prediction_year, start_prediction_year + TEST_YEARS_COUNT))
prediction_years

[2021, 2022, 2023]

In [188]:
# Колонки, которые сначала заполним для будущей таблицы
future_columns = [
    F.SpatialDimValueCode.value,
    F.Location.value,
    F.Period.value,
]

# Таблица-заглушка для данных за один год
future_data_dump = pd.DataFrame(columns=future_columns)
# Заполним код и название страны
future_data_dump[F.SpatialDimValueCode.value] = regions_and_locations_data[F.SpatialDimValueCode.value]
future_data_dump[F.Location.value] = regions_and_locations_data[F.Location.value]

future_data_dump.head()

,SpatialDimValueCode,Location,Period
0,AGO,Angola,NaN
1,BDI,Burundi,NaN
2,BEN,Benin,NaN
3,BFA,Burkina Faso,NaN
4,BWA,Botswana,NaN


In [189]:
# Заполним данные кодов и названий стран для каждого года
future_data = pd.DataFrame(columns=future_columns)

for year in prediction_years:
    future_year_data = future_data_dump.copy()
    future_year_data[F.Period.value] = year
    
    future_data = pd.concat([
        future_data,
        future_year_data,
    ], ignore_index=True)

# Убедимся, что заполнили данные за все года
future_data[F.Period.value].value_counts()

Period
2021    181
2022    181
2023    181
Name: count, dtype: int64

Добавим в таблицу данные с коэффициентом `NegativeCoef`.

In [190]:
# Таблица со значением поля NegativeCoef для каждой страны
negative_coef_data = data \
    .groupby(by=F.SpatialDimValueCode.value) \
    [[F.NegativeCoef.value]].first()\
    .reset_index()

negative_coef_data.head()

,SpatialDimValueCode,NegativeCoef
0,AFG,0.845913
1,AGO,0.837126
2,ALB,0.593019
3,ARE,0.597472
4,ARG,0.631922


In [191]:
# Добавим эти данные к основной таблице
future_data = future_data.merge(
    negative_coef_data,
    on=F.SpatialDimValueCode.value,
    how='left'
)

future_data.head()

,SpatialDimValueCode,Location,Period,NegativeCoef
0,AGO,Angola,2021,0.837126
1,BDI,Burundi,2021,0.857912
2,BEN,Benin,2021,0.848642
3,BFA,Burkina Faso,2021,0.861075
4,BWA,Botswana,2021,0.828255


Добавим в обе таблицы признак `SmoothingLifeExpectancy` со сглаженным значением целевой переменной \
(ожидаемой продолжительности жизни).

In [192]:
data, future_data = get_data_with_smoothing_target_feature(data, future_data)

# Переместим столбец с целевой переменной в конец таблицы
data = move_column_to_end_table(data, F.LifeExpectancy.value)

# Убедимся, что столбец добавился в обе таблицы
display(data.head(2))
display(future_data.head(2))

,SpatialDimValueCode,Location,Period,ImmunizationMean,NegativeCoef,GdpPerCapita,Sanitation,SmoothingLifeExpectancy,LifeExpectancy
0,AFG,Afghanistan,2000,25.0,0.845913,174.930991,20.97,53.696819,53.82
1,AFG,Afghanistan,2001,35.0,0.845913,138.706822,20.98,54.498708,53.91


,SpatialDimValueCode,Location,Period,NegativeCoef,SmoothingLifeExpectancy
0,AFG,Afghanistan,2021,0.845913,60.501268
1,AFG,Afghanistan,2022,0.845913,60.399577


Напишем функцию, которая заполнит сглаженными значениями предикторы.

In [193]:
def fill_field_with_smoothed_value(
    source_data: pd.DataFrame,
    future_data: pd.DataFrame,
    source_field_name: str,
    smoothing_field_name: str,
) -> pd.DataFrame:
    """Формирует столбец с прогнозом будущих значений

    Args:
        source_data (pd.DataFrame): исходная таблица с текущими данными
        future_data (pd.DataFrame): таблица с предполагаемыми будущими значениями, 
             которые получим путем сглаживания значений из текущих данных
        source_field_name (str): имя поля в исходной таблицы, 
            значения которого будем прогнозировать на будущее путем сглаживания
        smoothing_field_name (str): имя поля со сглаженным спрогнозированным значением

    Returns:
        pd.DataFrame: таблица future_data с добавленным в нее признаком smoothing_field_name
    """
    # Копируем таблицы, чтобы не мутировать их    
    source_data = source_data.copy()
    future_data = future_data.copy()
    
    # Поля для сортировки
    fields_for_sorting = [F.SpatialDimValueCode.value, F.Period.value]
    
    # На всякий случай сортируем таблицы, хотя они должны приходить отсортированные
    source_data.sort_values(by=fields_for_sorting, inplace=True)
    future_data.sort_values(by=fields_for_sorting, inplace=True)
    
    # Коды всех стран
    location_codes = get_location_codes(source_data)
    
    # Список с годами, для которых будем считать сглаженные значения
    years_future_list = sorted(list(future_data[F.Period.value].unique()))
    # Количество предсказаний, которое будем делать
    predictions_count = len(years_future_list)
    
    # Список столбцов для таблицы со сглаженными значениями
    smoothing_columns = [F.SpatialDimValueCode.value, F.Period.value, smoothing_field_name]
    # Таблица, в которую будем собирать сглаженные значения
    smoothing_data = pd.DataFrame(columns=smoothing_columns)
    
    # Для каждой страны посчитаем сглаженные значения
    for code in location_codes:
        # Временной ряд значений признака исходной таблицы
        location_time_series = get_location_time_series(source_data, code, source_field_name)
        
        # Получим объект с моделью
        exp_smoothing_model = ExponentialSmoothing(
            location_time_series, # тренировочные данные
            trend='add', # тип тренда - аддитивный
            damped_trend=True, # затухание тренда
        )
        
        # Обучим модель
        exp_smoothing_model_fit = exp_smoothing_model.fit(
            smoothing_level=0.85, # альфа - коэффициент сглаживания для уровня
            smoothing_trend=0.5, # beta - коэффициент сглаживания тренда
        )
        # Получим прогноз 
        forecast = exp_smoothing_model_fit.forecast(predictions_count)
        
        # Сформируем таблицу с полученным для страны прогнозом
        location_smoothing_data = pd.DataFrame({
            F.SpatialDimValueCode.value: code,
            F.Period.value: years_future_list,
            smoothing_field_name: forecast.values,
        })
        
        # Добавим значения прогноза, полученного для страны, в общую таблицу
        smoothing_data = pd.concat(
            [smoothing_data, location_smoothing_data],
            ignore_index=True
        )
    
    # Добавим столбец с полученными сглаженными значениями
    future_data = future_data.merge(
        smoothing_data,
        on=[F.SpatialDimValueCode.value, F.Period.value],
        how='left',
    )
    # Переведем столбцы в числовой тип
    future_data[F.Period.value] = future_data[F.Period.value].astype(int)
    future_data[smoothing_field_name] = future_data[smoothing_field_name].astype(float)
    
    return future_data

Заполним предикторы прогнозом со сглаженными значениями.

In [194]:
future_data = fill_field_with_smoothed_value(
    data,
    future_data,
    F.ImmunizationMean.value,
    F.ImmunizationMean.value,
)

future_data = fill_field_with_smoothed_value(
    data,
    future_data,
    F.GdpPerCapita.value,
    F.GdpPerCapita.value,
)

future_data = fill_field_with_smoothed_value(
    data,
    future_data,
    F.Sanitation.value,
    F.Sanitation.value,
)

future_data.head()

,SpatialDimValueCode,Location,Period,NegativeCoef,SmoothingLifeExpectancy,ImmunizationMean,GdpPerCapita,Sanitation
0,AFG,Afghanistan,2021,0.845913,60.501268,61.287843,509.819659,54.357740
1,AFG,Afghanistan,2022,0.845913,60.399577,60.222715,512.004269,56.044841
2,AFG,Afghanistan,2023,0.845913,60.311459,59.370612,513.794197,57.723506
3,AGO,Angola,2021,0.837126,62.838936,47.488205,1244.502607,52.225953
4,AGO,Angola,2022,0.837126,63.011744,45.747820,997.910059,52.681321


Переставим колонки, чтобы они шли в том же порядке, что в основной таблице.

In [195]:
ordered_columns = list(data.columns)
ordered_columns.remove(F.LifeExpectancy.value)

future_data = future_data.reindex(columns=ordered_columns)

future_data.head()

,SpatialDimValueCode,Location,Period,ImmunizationMean,NegativeCoef,GdpPerCapita,Sanitation,SmoothingLifeExpectancy
0,AFG,Afghanistan,2021,61.287843,0.845913,509.819659,54.357740,60.501268
1,AFG,Afghanistan,2022,60.222715,0.845913,512.004269,56.044841,60.399577
2,AFG,Afghanistan,2023,59.370612,0.845913,513.794197,57.723506,60.311459
3,AGO,Angola,2021,47.488205,0.837126,1244.502607,52.225953,62.838936
4,AGO,Angola,2022,45.747820,0.837126,997.910059,52.681321,63.011744


In [196]:
# Сохраним полученную таблицу для дальнейшего использования
future_data.to_csv('./app/data/future_data.csv', index=False)

## Создание таблицы с данными стран за прошлые годы

Эта таблица будет нужна для построения графика значений целевой переменной за прошлые года.

In [197]:
past_data = data[[
    F.SpatialDimValueCode.value,
    F.Location.value,
    F.Period.value,
    F.LifeExpectancy.value,
]]
past_data.head()

,SpatialDimValueCode,Location,Period,LifeExpectancy
0,AFG,Afghanistan,2000,53.82
1,AFG,Afghanistan,2001,53.91
2,AFG,Afghanistan,2002,55.15
3,AFG,Afghanistan,2003,56.09
4,AFG,Afghanistan,2004,56.48


In [198]:
# Сохраним таблицу
past_data.to_csv('./app/data/past_data.csv', index=False)

## Создание пайплайна для предсказания

Для пайплайна возьмем модель, которая показала лучшие результаты при исследовании.

In [199]:
# Поля, которые будут использоваться для предсказания
prediction_fields = [
    F.ImmunizationMean.value,
    F.NegativeCoef.value,
    F.GdpPerCapita.value,
    F.Sanitation.value,
    F.SmoothingLifeExpectancy.value,
]

# Создаём пайплайн
pipe = Pipeline([  
  ('MinMaxScaler', MinMaxScaler()),
  ('PolynomialFeatures', PolynomialFeatures(degree=3, include_bias=False)),
  ('AdaBoostRegressor', AdaBoostRegressor(
    base_estimator=Lasso(alpha=0.0003, max_iter=10000, random_state=RANDOM_STATE),
    learning_rate=0.0003,
    random_state=RANDOM_STATE
  ))
])

# Обучаем пайплайн
pipe.fit(data[prediction_fields], data[F.LifeExpectancy.value])

# Сериализуем pipeline и записываем результат в файл
with open('./app/model/pipeline.pkl', 'wb') as output:
    pickle.dump(pipe, output)

Проверим, на корректность сохраненный пайплайн.

In [201]:
# Десериализуем pipeline из файла
with open('./app/model/pipeline.pkl', 'rb') as pkl_file:
    pipe_from_file = pickle.load(pkl_file)

pipe_from_file

Pipeline(steps=[('MinMaxScaler', MinMaxScaler()),
                ('PolynomialFeatures',
                 PolynomialFeatures(degree=3, include_bias=False)),
                ('AdaBoostRegressor',
                 AdaBoostRegressor(base_estimator=Lasso(alpha=0.0003,
                                                        max_iter=10000,
                                                        random_state=42),
                                   learning_rate=0.0003, random_state=42))])

In [202]:
# Сравниваем предсказания исходного и восстановленного пайплайнов
all(pipe.predict(future_data[prediction_fields]) == pipe_from_file.predict(future_data[prediction_fields]))

True